
<div style="background-color:#CCCCCC; padding:12px; border-radius:8px;">
<h1 style="color:#003366; text-align:center; margin:8px 0;">Revisión y limpieza de 3 DataFrames (TPI - Data Analytics)</h1>
<p style="text-align:center; color:#003366; margin:0;"><em>Notebook docente en castellano — nombres descriptivos en snake_case — código y documentación</em></p>
</div>



<div style="background-color:#CCCCCC; padding:10px; border-radius:6px;">
<h2 style="color:black; text-align:center; margin-top:6px;">Resumen</h2>

<p style="color:black;">
Este notebook está diseñado con finalidades pedagógicas. Revisa, normaliza y valida tres datasets contenidos en CSV:
</p>

<ul style="color:black;">
<li><code>marketing.csv</code> → variable: <code>df_marketing</code></li>
<li><code>ventas.csv</code> → variable: <code>df_ventas</code></li>
<li><code>clientes.csv</code> → variable: <code>df_clientes</code></li>
</ul>

<p style="color:black;">
Coloca los CSV en <code>./data_in/</code> o en <code>/mnt/data/</code>. El notebook busca primero en <code>./data_in/</code> y si no encuentra, usa <code>/mnt/data/</code> (útil para entornos donde los archivos están pre-subidos).
</p>

</div>


In [1]:
# Imports y configuración inicial (nombres en castellano)
import os
from pathlib import Path
import json
import unicodedata
import zipfile

#!pip install gdown

import argparse
import pandas as pd
import numpy as np

import zipfile
from collections import defaultdict
from datetime import datetime
from math import isnan
ruta_base = ""


## 1. Crear un documento en Google Colaboratory y cargar los sets de datos como DataFrames

In [2]:
# Rutas: busca en ./data_in primero, si no existe usa /mnt/data

carpeta_entrada    = Path(ruta_base)
#carpeta_entrada_mnt   = Path('/mnt/data')

carpeta_reportes   = carpeta_entrada / 'reportes'
carpeta_reportes.mkdir(parents=True, exist_ok=True)

carpeta_limpios    = carpeta_entrada / 'limpios'
carpeta_limpios.mkdir(parents=True, exist_ok=True)

# Nombres esperados de archivos
archivo_ventas     = 'ventas.csv'
archivo_clientes   = 'clientes.csv'
archivo_marketing  = 'marketing.csv'

In [3]:
# --- Paso 3: Cargar archivos del curso ---
print("Cargando datasets del curso...")
ruta_ventas     = os.path.join(ruta_base, archivo_ventas)
ruta_clientes   = os.path.join(ruta_base, archivo_clientes)
ruta_marketing  = os.path.join(ruta_base, archivo_marketing)

df_ventas    = pd.read_csv(f"{ruta_ventas}")
df_clientes  = pd.read_csv(f"{ruta_clientes}")
df_marketing = pd.read_csv(f"{ruta_marketing}")

Cargando datasets del curso...


In [4]:
# Mostrar el DataFrame
print(f"""
df_ventas
{df_ventas}
{"*"*50}
df_clientes
{df_clientes}
{"*"*50}
df_marketing
{df_marketing}""")


df_ventas
      id_venta                producto   precio  cantidad fecha_venta  \
0          792       Cuadro decorativo   $69.94       5.0  02/01/2024   
1          811         Lámpara de mesa  $105.10       5.0  02/01/2024   
2         1156                Secadora   $97.96       3.0  02/01/2024   
3         1372                Heladera  $114.35       8.0  02/01/2024   
4         1546                Secadora  $106.21       4.0  02/01/2024   
...        ...                     ...      ...       ...         ...   
3030      1837         Horno eléctrico  $104.12       9.0  30/12/2024   
3031      2276                  Laptop   $85.27       9.0  30/12/2024   
3032      2696                  Laptop  $107.81       4.0  30/12/2024   
3033      2913              Smartphone   $99.85       7.0  30/12/2024   
3034      2930  Consola de videojuegos   $55.47       6.0  30/12/2024   

              categoria  
0            Decoración  
1            Decoración  
2     Electrodomésticos  
3     El

In [5]:
# ---------- Funciones utilitarias en castellano (snake_case) ----------
def sacar_acentos(texto):
    """
    Elimina acentos (tildes/diacríticos) de un texto.
    Mantiene NaN intactos.
    """
    if pd.isna(texto):
        return texto
    texto = str(texto)
    nk = unicodedata.normalize('NFKD', texto)
    return ''.join([c for c in nk if not unicodedata.combining(c)])

In [6]:
TOKENS_VALOR_FALTANTE = {
    '', 'na', 'n/a', 'null', 'none', 'sin dato', 's/d', 'nd', '-', '--', '?', 'sin_dato', 'n/d'
}

def es_valor_faltante(valor):
    """
    Determina si un valor debe considerarse faltante (True) usando tokens y NaN.
    """
    if pd.isna(valor):
        return True
    s = str(valor).strip().lower()
    s = sacar_acentos(s)
    return s in TOKENS_VALOR_FALTANTE

In [7]:
# ---------- Funciones de limpieza ----------
def aplicar_regla_columna(serie, regla):
    """
    Aplica una regla a una serie (columna).
    Firma compatible con la versión anterior (reemplaza la implementación previa).
    regla puede ser:
      - 'strip','lower','upper','title','quitar_acentos','numeric','date'
    o un tuple (tipo, opciones) con opciones:
      - numeric: remove_non_digits (bool), remove_thousands (bool), as_int (bool), thousands_separator (',' o '.')
      - date: formats (list), dayfirst (bool), format_output ('YYYY/MM/DD' para forzar cadena)
      - texto: normalizar_acentos (bool)
    Retorna la serie transformada (sin forzar dtype final).
    """
    tipo, opts = regla if isinstance(regla, tuple) else (regla, {})
    opts = opts or {}
    s = serie.copy()

    # -------- Texto y normalización de acentos opcional --------
    if tipo == 'strip':
        s = s.map(lambda x: str(x).strip() if not pd.isna(x) else x)
    elif tipo == 'lower':
        s = s.map(lambda x: str(x).strip().lower() if not pd.isna(x) else x)
        if opts.get('normalizar_acentos'):
            s = s.map(lambda x: sacar_acentos(x) if not pd.isna(x) else x)
    elif tipo == 'upper':
        s = s.map(lambda x: str(x).strip().upper() if not pd.isna(x) else x)
        if opts.get('normalizar_acentos'):
            s = s.map(lambda x: sacar_acentos(x) if not pd.isna(x) else x)
    elif tipo == 'title':
        s = s.map(lambda x: str(x).strip().title() if not pd.isna(x) else x)
        if opts.get('normalizar_acentos'):
            s = s.map(lambda x: sacar_acentos(x) if not pd.isna(x) else x)
    elif tipo == 'quitar_acentos':
        s = s.map(lambda x: sacar_acentos(x).strip() if not pd.isna(x) else x)

    # -------- Numeric robusto --------
    elif tipo == 'numeric':
        def to_num(v):
            if pd.isna(v):
                return np.nan
            t = str(v).strip()
            t = t.replace('$', '')
            if opts.get('remove_non_digits', False):
                t = ''.join([c for c in t if c.isdigit() or c in '.-'])
            if opts.get('remove_thousands', False):
                sep = opts.get('thousands_separator', ',')
                if sep == ',':
                    # Quitar puntos mil y comas de decimales no soportadas -> suponer coma miles y punto decimales
                    t = t.replace('.', '').replace(',', '')
                else:
                    t = t.replace(',', '').replace('.', '')   
            try:
                val = pd.to_numeric(t, errors='coerce')
                if opts.get('as_int', False):
                    if pd.isna(val):
                        return pd.NA
                    try:
                        # intentar entero simple
                        return int(val)
                    except Exception:
                        return pd.NA
                return float(val) if not pd.isna(val) else np.nan
            except Exception:
                return np.nan
        s = s.map(to_num)

    # -------- Fecha robusta --------
    elif tipo == 'date':
        formatos = opts.get('formats', [])
        dayfirst = opts.get('dayfirst', True)
        def to_date(v):
            if pd.isna(v):
                return pd.NaT
            t = str(v).strip()
            # Probar formatos explícitos
            for fmt in formatos:
                try:
                    return pd.to_datetime(datetime.strptime(t, fmt))
                except Exception:
                    continue
            # Fallback: pandas con dayfirst
            try:
                return pd.to_datetime(t, dayfirst=dayfirst, errors='coerce')
            except Exception:
                return pd.NaT
        s = s.map(to_date)

    else:
        # Default: trim y convertir tokens faltantes a NaN para texto
        if s.dtype == object or pd.api.types.is_string_dtype(s):
            s = s.map(lambda x: np.nan if es_valor_faltante(x) else (str(x).strip() if not pd.isna(x) else x))
    return s

In [8]:
def convertir_tipos_postprocesamiento(df, reglas_por_columna):
    """
    Garantiza dtypes correctos:
     - Para columnas numeric: pd.to_numeric(...) + conversión a Int64 nullable si as_int, o float.
     - Para columnas date: intenta parsear según formatos; deja datetime64[ns] o, si 'format_output'=='YYYY/MM/DD', devuelve strings con ese formato.
    """
    df2 = df.copy()
    for col, regla in (reglas_por_columna or {}).items():
        tipo = regla if not isinstance(regla, tuple) else regla[0]
        opts = {} if not isinstance(regla, tuple) else regla[1] or {}
        if tipo == 'numeric':
            df2[col] = pd.to_numeric(df2[col], errors='coerce')
            if opts.get('as_int', False):
                # convertir a Int64 nullable
                try:
                    df2[col] = df2[col].astype('Int64')
                except Exception:
                    # fallback: mantener float si conversion falla
                    df2[col] = pd.to_numeric(df2[col], errors='coerce')
        elif tipo == 'date':
            formatos = opts.get('formats', [])
            dayfirst = opts.get('dayfirst', True)
            parsed = pd.Series(pd.NaT, index=df2.index)
            # probar formatos explícitos uno por uno
            for fmt in formatos:
                try:
                    mask_necesita = parsed.isna()
                    intent = pd.to_datetime(df2.loc[mask_necesita, col].astype(str), format=fmt, errors='coerce')
                    parsed.loc[mask_necesita] = intent
                except Exception:
                    pass
            # fallback general para los que quedaron NaT
            still_na = parsed.isna()
            if still_na.any():
                parsed.loc[still_na] = pd.to_datetime(df2.loc[still_na, col].astype(str), dayfirst=dayfirst, errors='coerce')
            df2[col] = parsed
            if opts.get('format_output') == 'YYYY/MM/DD':
                # convertir a string con formato pedido (mantener NaT como NaN)
                df2[col] = df2[col].dt.strftime('%Y/%m/%d')
    for col in df2.select_dtypes(include=['object']).columns:
        df2[col] = df2[col].map(sacar_acentos)
    return df2

In [9]:
def limpiar_dataframe(df, reglas_por_columna=None):
    """
    Limpieza principal (misma firma que antes):
     1) Aplica aplicar_regla_columna por cada columna según reglas_por_columna
     2) Para columnas texto por defecto: strip + tokens faltantes -> NaN
     3) Elimina duplicados exactos
     4) Convierte tipos numéricos y fechas con convertir_tipos_postprocesamiento
    Retorna df limpio con dtypes corregidos.

    Aplica reglas de limpieza columna por columna.
    Si una columna no existe en el DataFrame, se omite con aviso educativo.
    """
    #reglas_por_columna = reglas_por_columna or {}

    reglas_por_columna = reglas_por_columna or {}
    df2 = df.copy()
    df2.columns = [str(c).strip() for c in df2.columns]

    for col in df2.columns:
        if col in reglas_por_columna:
            # usar la función aplicar_regla_columna existente en el notebook
            df2[col] = aplicar_regla_columna(df2[col], reglas_por_columna[col])
        else:
            if df2[col].dtype == object or pd.api.types.is_string_dtype(df2[col]):
                df2[col] = df2[col].map(lambda x: np.nan if es_valor_faltante(x) else (str(x).strip() if not pd.isna(x) else x))

    # eliminar duplicados exactos
    df2 = df2.drop_duplicates(keep='first').reset_index(drop=True)

    # intentos de post-conversión sencillos para numeric/date según reglas:
    for col, regla in (reglas_por_columna or {}).items():
        tipo = regla if not isinstance(regla, tuple) else regla[0]
        opts = {} if not isinstance(regla, tuple) else regla[1] or {}
        if tipo == 'numeric':
            df2[col] = pd.to_numeric(df2[col], errors='coerce')
            if opts.get('as_int', False):
                try:
                    df2[col] = df2[col].astype('Int64')
                except Exception:
                    pass
        elif tipo == 'date':
            formatos = opts.get('formats', [])
            dayfirst = opts.get('dayfirst', True)
            parsed = pd.Series(pd.NaT, index=df2.index)
            for fmt in formatos:
                try:
                    mask = parsed.isna()
                    parsed.loc[mask] = pd.to_datetime(df2.loc[mask, col].astype(str), format=fmt, errors='coerce')
                except Exception:
                    pass
            still_na = parsed.isna()
            if still_na.any():
                parsed.loc[still_na] = pd.to_datetime(df2.loc[still_na, col].astype(str), dayfirst=dayfirst, errors='coerce')
            # si se pidió format_output, devolver cadena con YYYY/MM/DD
            if opts.get('format_output') == 'YYYY/MM/DD':
                df2[col] = parsed.dt.strftime('%Y/%m/%d')
            else:
                df2[col] = parsed

    return df2

In [10]:

# Detección de outliers (IQR) - función corregida y robusta
def mascara_valores_atipicos_rango_intercuartil(serie_datos):
    """
    Devuelve una tupla: (mascara_bool_series, cantidad_outliers, (limite_inferior, limite_superior))
    - serie_datos: pd.Series (acepta valores no numéricos, se intentará convertir)
    - La máscara tiene la misma indexación que la serie original (NaNs -> False)
    """
    # Intentar convertir a numérico (coerce -> NaN para no numéricos)
    serie_numerica = pd.to_numeric(serie_datos, errors='coerce')
    # Serie limpia para cálculos de cuartiles (sin NaN)
    serie_limpia = serie_numerica.dropna().astype(float)
    if serie_limpia.shape[0] < 4:
        # No hay suficientes datos para IQR: devolver máscara False de la misma longitud
        mascara = pd.Series([False] * len(serie_datos), index=serie_datos.index)
        return mascara, int(mascara.sum()), (None, None)
    cuartil_1 = float(serie_limpia.quantile(0.25))
    cuartil_3 = float(serie_limpia.quantile(0.75))
    rango_intercuartil = cuartil_3 - cuartil_1
    limite_inferior = cuartil_1 - 1.5 * rango_intercuartil
    limite_superior = cuartil_3 + 1.5 * rango_intercuartil
    # Crear máscara sobre la serie numérica original (alineada con el index original)
    mascara = (serie_numerica < limite_inferior) | (serie_numerica > limite_superior)
    mascara = mascara.fillna(False).astype(bool)
    return mascara, int(mascara.sum()), (limite_inferior, limite_superior)


In [11]:
# Detección de outliers (Z-score)
def mascara_valores_atipicos_zscore(serie_datos, umbral=3.0):
    """
    Devuelve máscara booleana (True = outlier) según Z-score.
    """
    serie_limpia = serie_datos.dropna().astype(float)
    if serie_limpia.shape[0] < 4 or serie_limpia.std() == 0:
        return pd.Series([False] * len(serie_datos), index=serie_datos.index)
    puntaje_z = (serie_datos - serie_limpia.mean()) / serie_limpia.std()
    return puntaje_z.abs() > umbral    

In [12]:
print('Funciones utilitarias definidas.')

Funciones utilitarias definidas.


# limpieza y normalización

In [13]:
# ---------- Detección de problemas en un DataFrame ----------
def detectar_problemas_en_dataframe(df: pd.DataFrame):
    
    resumen = {}
    resumen['filas'] = df.shape[0]
    resumen['columnas'] = df.shape[1]
    resumen['nulos_por_columna'] = df.isna().sum().to_dict()
    dup_mask = df.duplicated(keep=False)
    resumen['duplicados_exactos'] = int(dup_mask.sum())

    chequeos_por_columna = {}
    for col in df.columns:
        serie = df[col]
        info = {'dtype': str(serie.dtype), 'nulos': int(serie.isna().sum())}
        # texto
        if serie.dtype == object or pd.api.types.is_string_dtype(serie):
            s = serie.astype(str)
            info['espacios_inicio'] = int(s.str.match(r'^\s+').sum())
            info['espacios_final'] = int(s.str.match(r'\s+$').sum())
            try:
                unique_original = set(s.dropna().unique())
                unique_lower = set(s.dropna().str.lower().unique())
                info['unique_original'] = len(unique_original)
                info['unique_lower'] = len(unique_lower)
                info['variantes_mayusculas'] = len(unique_lower) < len(unique_original)
            except Exception:
                info['unique_original'] = serie.nunique(dropna=True)
                info['unique_lower'] = None
                info['variantes_mayusculas'] = None
            try:
                unaccented = s.dropna().map(lambda x: sacar_acentos(x).lower())
                groups = unaccented.groupby(unaccented).size()
                conflicts = groups[groups > 1]
                info['grupos_variantes_acentos'] = int(conflicts.shape[0])
                ejemplos = {}
                if not conflicts.empty:
                    for val in conflicts.index[:5]:
                        originales = sorted(list(s[unaccented == val].unique())[:10])
                        ejemplos[val] = originales
                info['ejemplos_variantes_acentos'] = ejemplos
            except Exception:
                info['grupos_variantes_acentos'] = None
                info['ejemplos_variantes_acentos'] = {}
            info['tokens_aparente_faltante'] = int(
                s.map(lambda x: str(x).strip().lower()).map(lambda v: sacar_acentos(v) in TOKENS_VALOR_FALTANTE).sum()
            )
            info['muestras'] = list(s.dropna().unique()[:10])
        else:
            # numeric
            if pd.api.types.is_numeric_dtype(serie) or (serie.dropna().astype(str).str.replace('.','',1).str.isnumeric().all() if len(serie.dropna())>0 else False):
                try:
                    serie_numerica = serie.dropna().astype(float)
                except Exception:
                    serie_numerica = pd.to_numeric(serie, errors='coerce').dropna().astype(float)
                info['media'] = float(serie_numerica.mean()) if not serie_numerica.empty else None
                info['std'] = float(serie_numerica.std()) if not serie_numerica.empty else None
                info['min'] = float(serie_numerica.min()) if not serie_numerica.empty else None
                info['max'] = float(serie_numerica.max()) if not serie_numerica.empty else None
               
                if len(serie_numerica) >= 4:
                    cuartil_1 = serie_numerica.quantile(0.25)
                    cuartil_3 = serie_numerica.quantile(0.75)
                    rango_intercuartil = cuartil_3 - cuartil_1
                    limite_inferior = cuartil_1 - 1.5 * rango_intercuartil
                    limite_superior = cuartil_3 + 1.5 * rango_intercuartil
                    mascara_outliers_iqr,info['outliers_iqr'] ,info['limites_iqr'] =mascara_valores_atipicos_rango_intercuartil(serie)
                    
                else:
                    info['outliers_iqr'] = None
                    info['limites_iqr'] = None
                if len(serie_numerica) >= 4 and serie_numerica.std() != 0:
                    z = (serie_numerica - serie_numerica.mean()) / serie_numerica.std()
                    info['outliers_z'] = int((z.abs() > 3).sum())
                else:
                    info['outliers_z'] = None
            else:
                # fechas intento parseo
                parsed = pd.to_datetime(serie, errors='coerce', dayfirst=True)
                info['fechas_parseables'] = int(parsed.notna().sum())
                info['muestras'] = list(serie.dropna().unique()[:10])
        chequeos_por_columna[col] = info

    filas_problemas = []
    for idx, fila in df.iterrows():
        lista_problemas = []
        if dup_mask.loc[idx]:
            lista_problemas.append('duplicado_exacto')
        for col in df.columns:
            val = fila[col]
            # heurísticas textuales
            if pd.api.types.is_string_dtype(type(val)) or isinstance(val, str) or (
                not pd.isna(val) and not pd.api.types.is_numeric_dtype(type(val)) and str(chequeos_por_columna[col].get('dtype','')).startswith('object')
            ):
                s = str(val)
                if s != s.strip():
                    lista_problemas.append(f'espacios_en_columna_{col}')
                if chequeos_por_columna[col].get('variantes_mayusculas'):
                    if s and s != s.lower() and s.lower() in [str(x).lower() for x in df[col].dropna().unique()]:
                        lista_problemas.append(f'inconsistencia_mayusculas_columna_{col}')
                if chequeos_por_columna[col].get('grupos_variantes_acentos') and chequeos_por_columna[col]['grupos_variantes_acentos'] > 0:
                    try:
                        un = sacar_acentos(s).lower()
                        group_vals = [x for x in chequeos_por_columna[col].get('muestras', []) if sacar_acentos(str(x)).lower() == un]
                        if group_vals and any(sacar_acentos(str(x)).lower() != sacar_acentos(s).lower() for x in group_vals):
                            lista_problemas.append(f'variantes_acentos_columna_{col}')
                    except Exception:
                        pass
                if es_valor_faltante(s):
                    lista_problemas.append(f'token_faltante_columna_{col}')
            else:
                # heurísticas numéricas
                try:
                    fval = float(val)
                    info_col = chequeos_por_columna[col]
                    limites = info_col.get('limites_iqr')
                    if limites and (fval < limites[0] or fval > limites[1]):
                        lista_problemas.append(f'outlier_iqr_columna_{col}')
                    if info_col.get('std') not in (None, 0):
                        mean = info_col.get('media')
                        std = info_col.get('std')
                        if std and abs((fval - mean) / std) > 3:
                            lista_problemas.append(f'outlier_z_columna_{col}')
                except Exception:
                    pass
        if lista_problemas:
            filas_problemas.append({
                'row_index': idx,
                'problemas': ';'.join(sorted(set(lista_problemas))),
                'muestra': json.dumps({str(c): str(fila[c]) for c in df.columns[:8]})
            })

    df_problemas = pd.DataFrame(filas_problemas)
    return resumen, chequeos_por_columna, df_problemas

print('Función detectar_problemas_en_dataframe cargada.')

Función detectar_problemas_en_dataframe cargada.


In [14]:
# Función auxiliar para guardar CSVs
def guardar_csv(df, ruta):
    """
    Guarda df en ruta (string o Path). Crea directorio padre si no existe.
    """
    ruta = Path(ruta)
    ruta.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(ruta, index=False, encoding='utf-8')
    return ruta

In [15]:
dic_frames = {
    ruta_ventas     : df_ventas,
    ruta_clientes   : df_clientes,
    ruta_marketing  : df_marketing
}

In [16]:
# ---------- Proceso principal para los 3 CSV ----------
#id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
reglas_marketing = {
                        'producto': ('lower', {'normalizar_acentos': True}),
                        'canal': ('lower', {'normalizar_acentos': True}),
                        'costo': ('numeric', {'remove_thousands': True, 'as_int': False}),
                        'fecha_inicio': ('date', {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']}),
                        'fecha_fin': ('date', {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']})
}
# id_venta,producto,precio,cantidad,fecha_venta,categoria
reglas_ventas = {
                        'producto': ('lower', {'normalizar_acentos': True}),
                        'precio': ('numeric', {'remove_thousands': True, 'as_int': False}),
                        'cantidad': ('numeric', {'remove_thousands': True, 'as_int': False}),
                        'fecha_venta': ('date', {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']}),
                        'categoria': ('lower', {'normalizar_acentos': True})
}
#id_cliente,nombre,edad,ciudad,ingresos
reglas_clientes = {
                        'nombre': ('title', {'normalizar_acentos': True}),
                        'edad': ('numeric', {'remove_thousands': True, 'as_int': True}),
                        'ciudad': ('title', {'normalizar_acentos': True}),
                        'ingresos': ('numeric', {'remove_thousands': True, 'as_int': False})
}

reglas_por_archivo = {
    'marketing.csv': reglas_marketing,
    'ventas.csv'   : reglas_ventas,
    'clientes.csv' : reglas_clientes
}
reportes_creados = []
zip_path = carpeta_reportes.parent / 'reports_dataset_tpi_v2.zip'


In [17]:
# ---------- Bucle/menu principal (usa dic_frames) ----------
def menu_procesar_diccionario(dic_frames, reglas_por_archivo):
    """
    Recorre dic_frames: clave = nombre_archivo (ej. 'marketing.csv'), valor = DataFrame.
    Ejecuta: detectar_problemas_en_dataframe antes, limpiar_dataframe, detectar_problemas_en_dataframe despues,
    imprime resúmenes y guarda cleaned en carpeta_limpios con sufijo ' limpio.csv'.
    También sobreescribe variables en RAM (df_marketing, df_ventas, df_clientes) si se encuentran en el nombre.
    """
    # trabajamos sobre una copia para evitar modificar dict original por error
    for nombre_archivo, df_actual in dic_frames.items():
        resumen_antes, chequeos_antes, problemas_antes = detectar_problemas_en_dataframe(df_actual)
        print(f"--- RESUMEN ANTES: {nombre_archivo} ---")
        print(resumen_antes)
        # Para no volcar objetos muy grandes, mostramos el head del DataFrame de problemas (si existe)
        print('Muestras de problemas antes (primeras 5 filas):')
        display(problemas_antes.head(5) if not problemas_antes.empty else 'No se detectaron filas con problemas.')

        reglas = reglas_por_archivo.get(nombre_archivo, {})
        df_limpio = limpiar_dataframe(df_actual, reglas_por_columna=reglas)

        resumen_despues, chequeos_despues, problemas_despues = detectar_problemas_en_dataframe(df_limpio)
        print(f"n--- RESUMEN DESPUÉS: {nombre_archivo} ---")
        print(resumen_despues)
        print('Muestras de problemas después (primeras 5 filas):')
        display(problemas_despues.head(5) if not problemas_despues.empty else 'No se detectaron filas con problemas tras la limpieza.')

        # mostrar separadores y tipo-nombre
        print('-'*100)
        print(f'nombre_archivo = {nombre_archivo}')
        print(f'type(nombre_archivo) = {type(nombre_archivo)}')
        print('-'*100)

        ruta_nombre_limpio = nombre_archivo[:-4] + '_limpio.csv' if nombre_archivo.lower().endswith('.csv') else nombre_archivo + ' limpio.csv'
        print(f'ruta_nombre_limpio = {ruta_nombre_limpio}')

        # guardar cleaned
        ruta_guardado = carpeta_limpios / ruta_nombre_limpio
        guardar_csv(df_limpio, ruta_guardado)
        print(f'Guardado cleaned en: {ruta_guardado}')

        # sobreescribir en RAM según el nombre
        # (nota: usar globals() para actualizar variables en el entorno global del notebook)
        if 'marketing' in nombre_archivo.lower():
            globals()['df_marketing'] = df_limpio
        elif 'ventas' in nombre_archivo.lower():
            globals()['df_ventas'] = df_limpio
        elif 'clientes' in nombre_archivo.lower():
            globals()['df_clientes'] = df_limpio

    print('Proceso completo del diccionario de DataFrames.')

In [18]:
menu_procesar_diccionario(dic_frames, reglas_por_archivo)

--- RESUMEN ANTES: ventas.csv ---
{'filas': 3035, 'columnas': 6, 'nulos_por_columna': {'id_venta': 0, 'producto': 0, 'precio': 2, 'cantidad': 2, 'fecha_venta': 0, 'categoria': 0}, 'duplicados_exactos': 70}
Muestras de problemas antes (primeras 5 filas):


,row_index,problemas,muestra
0,820,duplicado_exacto,"{""id_venta"": ""56"", ""producto"": ""Cortinas"", ""pr..."
1,821,duplicado_exacto,"{""id_venta"": ""421"", ""producto"": ""L\u00e1mpara ..."
2,822,duplicado_exacto,"{""id_venta"": ""424"", ""producto"": ""Jarr\u00f3n d..."
3,823,duplicado_exacto,"{""id_venta"": ""1868"", ""producto"": ""Cafetera"", ""..."
4,824,duplicado_exacto,"{""id_venta"": ""2545"", ""producto"": ""Auriculares""..."


n--- RESUMEN DESPUÉS: ventas.csv ---
{'filas': 3000, 'columnas': 6, 'nulos_por_columna': {'id_venta': 0, 'producto': 0, 'precio': 2, 'cantidad': 2, 'fecha_venta': 0, 'categoria': 0}, 'duplicados_exactos': 0}
Muestras de problemas después (primeras 5 filas):


'No se detectaron filas con problemas tras la limpieza.'

----------------------------------------------------------------------------------------------------
nombre_archivo = ventas.csv
type(nombre_archivo) = <class 'str'>
----------------------------------------------------------------------------------------------------
ruta_nombre_limpio = ventas_limpio.csv
Guardado cleaned en: limpios\ventas_limpio.csv
--- RESUMEN ANTES: clientes.csv ---
{'filas': 567, 'columnas': 5, 'nulos_por_columna': {'id_cliente': 0, 'nombre': 0, 'edad': 0, 'ciudad': 0, 'ingresos': 0}, 'duplicados_exactos': 0}
Muestras de problemas antes (primeras 5 filas):


...

----------------------------------------------------------------------------------------------------
nombre_archivo = clientes.csv
type(nombre_archivo) = <class 'str'>
----------------------------------------------------------------------------------------------------
ruta_nombre_limpio = clientes_limpio.csv
Guardado cleaned en: limpios\clientes_limpio.csv
--- RESUMEN ANTES: marketing.csv ---
{'filas': 90, 'columnas': 6, 'nulos_por_columna': {'id_campanha': 0, 'producto': 0, 'canal': 0, 'costo': 0, 'fecha_inicio': 0, 'fecha_fin': 0}, 'duplicados_exactos': 0}
Muestras de problemas antes (primeras 5 filas):


,row_index,problemas,muestra
0,41,outlier_iqr_columna_costo,"{""id_campanha"": ""88"", ""producto"": ""Alfombra"", ..."


n--- RESUMEN DESPUÉS: marketing.csv ---
{'filas': 90, 'columnas': 6, 'nulos_por_columna': {'id_campanha': 0, 'producto': 0, 'canal': 0, 'costo': 0, 'fecha_inicio': 0, 'fecha_fin': 0}, 'duplicados_exactos': 0}
Muestras de problemas después (primeras 5 filas):


,row_index,problemas,muestra
0,1,outlier_iqr_columna_costo,"{""id_campanha"": ""12"", ""producto"": ""tablet"", ""c..."
1,13,outlier_iqr_columna_costo,"{""id_campanha"": ""34"", ""producto"": ""heladera"", ..."
2,28,outlier_iqr_columna_costo,"{""id_campanha"": ""49"", ""producto"": ""cafetera"", ..."
3,32,outlier_iqr_columna_costo,"{""id_campanha"": ""72"", ""producto"": ""tablet"", ""c..."
4,39,outlier_iqr_columna_costo,"{""id_campanha"": ""53"", ""producto"": ""espejo deco..."


----------------------------------------------------------------------------------------------------
nombre_archivo = marketing.csv
type(nombre_archivo) = <class 'str'>
----------------------------------------------------------------------------------------------------
ruta_nombre_limpio = marketing_limpio.csv
Guardado cleaned en: limpios\marketing_limpio.csv
Proceso completo del diccionario de DataFrames.


In [19]:

# ======= Celda automática: generar reporte consolidado en Excel =======
# Esta celda crea un archivo Excel en carpeta_reportes con hojas:
# - duplicados
# - outliers
# - resumen_filtros
import pandas as pd
from pathlib import Path

# Asegurarse que carpeta_reportes existe
try:
    carpeta_reportes.mkdir(parents=True, exist_ok=True)
except Exception:
    carpeta_reportes = Path('/mnt/data/reportes')
    carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Intentar obtener DataFrames limpios desde el entorno; si no existen, cargar desde rutas
dfs_por_origen = {}
try:
    dfs_por_origen['ventas'] = df_ventas
except NameError:
    try:
        dfs_por_origen['ventas'] = pd.read_csv(ruta_ventas)
    except Exception:
        dfs_por_origen['ventas'] = pd.DataFrame()

try:
    dfs_por_origen['clientes'] = df_clientes
except NameError:
    try:
        dfs_por_origen['clientes'] = pd.read_csv(ruta_clientes)
    except Exception:
        dfs_por_origen['clientes'] = pd.DataFrame()

try:
    dfs_por_origen['marketing'] = df_marketing
except NameError:
    try:
        dfs_por_origen['marketing'] = pd.read_csv(ruta_marketing)
    except Exception:
        dfs_por_origen['marketing'] = pd.DataFrame()

# Listas para consolidar
lista_duplicados = []
lista_outliers = []
lista_resumen = []

for nombre, df_actual in dfs_por_origen.items():
    if df_actual is None or df_actual.empty:
        lista_resumen.append({'dataset': nombre, 'filas_iniciales': 0, 'duplicados_encontrados': 0, 'outliers_encontrados': 0, 'filas_finales': 0})
        continue

    filas_iniciales = df_actual.shape[0]

    # Duplicados por fila completa (keep=False)
    duplicados_df = df_actual[df_actual.duplicated(keep=False)].copy()
    if not duplicados_df.empty:
        duplicados_df['dataset'] = nombre
        lista_duplicados.append(duplicados_df)

    # Outliers: aplicar IQR por cada columna numérica
    outliers_encontrados_dataset = []
    for columna in df_actual.columns:
        try:
            mascara, cantidad, limites = mascara_valores_atipicos_rango_intercuartil(df_actual[columna])
            if cantidad > 0:
                temp = df_actual[mascara].copy()
                temp['columna_outlier'] = columna
                temp['dataset'] = nombre
                lista_outliers.append(temp)
                outliers_encontrados_dataset.append((columna, cantidad))
        except Exception:
            # Si la detección falla en una columna, continuar con la siguiente
            continue

    # Filas finales (suponiendo que se eliminarían duplicados y outliers)
    filas_sin_duplicados = df_actual.drop_duplicates().shape[0]
    filas_finales_estimadas = max(0, filas_sin_duplicados - sum([c for _, c in outliers_encontrados_dataset]))

    lista_resumen.append({
        'dataset': nombre,
        'filas_iniciales': int(filas_iniciales),
        'duplicados_encontrados': int(duplicados_df.shape[0]) if not duplicados_df.empty else 0,
        'outliers_encontrados': sum([c for _, c in outliers_encontrados_dataset]),
        'filas_finales': int(filas_finales_estimadas)
    })

# Consolidar resultados
df_duplicados_total = pd.concat(lista_duplicados, ignore_index=True) if lista_duplicados else pd.DataFrame()
df_outliers_total = pd.concat(lista_outliers, ignore_index=True) if lista_outliers else pd.DataFrame()
df_resumen = pd.DataFrame(lista_resumen)

# Guardar en un solo Excel con tres hojas
ruta_excel = carpeta_reportes / 'reporte_limpieza.xlsx'
with pd.ExcelWriter(ruta_excel, engine='xlsxwriter') as writer:
    try:
        df_duplicados_total.to_excel(writer, index=False, sheet_name='duplicados')
    except Exception:
        pd.DataFrame().to_excel(writer, index=False, sheet_name='duplicados')
    try:
        df_outliers_total.to_excel(writer, index=False, sheet_name='outliers')
    except Exception:
        pd.DataFrame().to_excel(writer, index=False, sheet_name='outliers')
    try:
        df_resumen.to_excel(writer, index=False, sheet_name='resumen_filtros')
    except Exception:
        pd.DataFrame(lista_resumen).to_excel(writer, index=False, sheet_name='resumen_filtros')

print("Archivo generado en:", ruta_excel)


ModuleNotFoundError: No module named 'xlsxwriter'